In [22]:
import glob
from os import path, makedirs
from PIL import Image
import rasterio
import numpy as np
from tqdm.notebook import tqdm
import json

In [23]:
root_path = "/data/b2p-siteident"
rgb_tile_path = "data/rgb_tiles/"

In [24]:
for fp in glob.glob(path.join(root_path, rgb_tile_path, "*")):
    if fp.endswith("_jpg"):
        continue
    print(fp)
    if not path.isdir(fp + "_jpg"):
        makedirs(fp + "_jpg")

/data/b2p-siteident/data/rgb_tiles/v2_b2p_rgb_large_1150_600
/data/b2p-siteident/data/rgb_tiles/v1_b2p_rgb_large_590_320
/data/b2p-siteident/data/rgb_tiles/v1_b2p_rgb_large_1150_600
/data/b2p-siteident/data/rgb_tiles/v2_b2p_rgb_large_2350_1200
/data/b2p-siteident/data/rgb_tiles/v1_b2p_rgb_large_2350_1200
/data/b2p-siteident/data/rgb_tiles/v2_b2p_rgb_large_590_320


In [25]:
for fp in glob.glob(path.join(root_path, rgb_tile_path, "*")):
    if fp.endswith("_jpg"):
        continue
    out_fp = fp + "_jpg"
    for tif_fp in tqdm(glob.glob(path.join(fp, "*.tif")), desc=fp.split("/")[-1]):
        out_file = tif_fp.replace(fp, out_fp).replace(".tif", ".jpg")
        if not path.isfile(out_file):
            with rasterio.open(tif_fp) as src:
                arr = np.concatenate((
                    np.expand_dims(src.read(1), -1),
                    np.expand_dims(src.read(2), -1),
                    np.expand_dims(src.read(3), -1),
                ), -1)
                img = Image.fromarray(arr)
                img.save(out_file)

v2_b2p_rgb_large_1150_600:   0%|          | 0/3172 [00:00<?, ?it/s]

v1_b2p_rgb_large_590_320:   0%|          | 0/3354 [00:00<?, ?it/s]

v1_b2p_rgb_large_1150_600:   0%|          | 0/3354 [00:00<?, ?it/s]

v2_b2p_rgb_large_2350_1200:   0%|          | 0/3396 [00:00<?, ?it/s]

v1_b2p_rgb_large_2350_1200:   0%|          | 0/3354 [00:00<?, ?it/s]

v2_b2p_rgb_large_590_320:   0%|          | 0/3396 [00:00<?, ?it/s]

In [26]:
rgb_tiles_paths = "../data/rgb_tiles/*_jpg"

stats = {}

for fp in glob.glob(rgb_tiles_paths):
    name = fp.split("/")[-1]
    print(name)
    channel_sum = np.zeros(3)
    count = 0
    for img_fp in glob.glob(path.join(fp, "*.jpg")):
        if "-te-" in img_fp or "test" in img_fp:
            continue
        img = plt.imread(img_fp)
        channel_sum += (img / 255).sum(0).sum(0)
        count += img.shape[0] * img.shape[1]
    mean = (channel_sum / count).tolist()

    channel_sq_sum = np.zeros(3)
    for img_fp in glob.glob(path.join(fp, "*.jpg")):
        img = plt.imread(img_fp)
        channel_sq_sum += np.power(img / 255 - mean, 2).sum(0).sum(0)
        count += img.shape[0] * img.shape[1]
    std = (channel_sq_sum / count).tolist()
    stats[name] = {
        "mean": mean,
        "std": std
    }

v1_b2p_rgb_large_590_320_jpg
v1_b2p_rgb_large_1150_600_jpg
v2_b2p_rgb_large_2350_1200_jpg
v2_b2p_rgb_large_590_320_jpg
v1_b2p_rgb_large_2350_1200_jpg
v2_b2p_rgb_large_1150_600_jpg


In [27]:
with open("../data/rgb_tiles/stats.json", "w+") as f:
    json.dump(stats, f)